<a href="https://colab.research.google.com/github/punkmic/pytorch_neural_network_mnist/blob/master/neural_network_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pytorch - Neural Network MNIST**

In [1]:
import numpy as np
import click
try:
  import torch
  from torch import nn, optim
  from torchvision import datasets, transforms
except:
  !pip install torch

In [9]:
# @click.command()
# @click.option('--input_size', default=784, help='Size of the input layer of the model.')
# @click.option('--output_size', default=10, help='Size of the output layer of the model.')
def create_model(input_size, output_size):
    """
    This function creates a sequential feedforward neural network model for image classification.

    Args:
        input_size (int, optional): Size of the input to the model. In this case, the input size is 784,
        representing the size of a 28x28 image after being flattened. Defaults to 784.
        output_size (int, optional): Number of classes that the model should predict. In this case, 
        the output size is 10, representing the number of digits from 0-9. Defaults to 10.

    Returns:
        A PyTorch `nn.Sequential` object representing the model.
    """
    return nn.Sequential(nn.Linear(input_size, 228),
                  nn.ReLU(),
                  nn.Linear(228, 64),
                  nn.ReLU(),
                  nn.Linear(64, output_size),
                  nn.LogSoftmax(dim=1)
                 )

In [3]:
def train(model, train_loader, cost, optimizer, epoch):
  """
    Train a machine learning model.
    
    Parameters:
    model (nn.Module): The model to be trained.
    train_loader (DataLoader): The training data loader.
    cost (function): The cost function used to evaluate the model's performance during training.
    optimizer (Optimizer): The optimization algorithm used to update the model's parameters.
    epoch (int): The number of iterations over the entire training data.
    
    Returns:
    None
  """
  # Loop through each epoch
  for e in range(epoch):
    # Initialize running loss to 0
    running_loss = 0
    # Initialize the number of correct predictions to 0
    correct = 0
    # Loop through each batch in the train_loader
    for data, target in train_loader:
      # Reshape the data tensor to have shape (batch_size, -1)
      data = data.view(data.shape[0], -1)
      # Zero the gradients of the model parameters
      optimizer.zero_grad()
      # Get the predictions from the model
      pred = model(data)
      # Calculate the loss between the predictions and the target
      loss = cost(pred, target)
      # Add the loss to the running total of the loss
      running_loss += loss
      # Compute the gradients of the loss with respect to the model parameters
      loss.backward()
      # Update the model parameters using the optimizer
      optimizer.step()
      # Find the index of the maximum prediction for each sample in the batch
      pred = pred.argmax(dim=1, keepdim=True)
      # Increase the number of correct predictions
      correct += pred.eq(target.view_as(pred)).sum().item()
    # Print the average loss and accuracy over the entire training dataset at the end of each epoch
    print(f"Epoch {e}: Loss {running_loss/len(train_loader.dataset)}, Accuracy {100*(correct/len(train_loader.dataset))}%")

In [ ]:
import pickle
import click

@click.command()
@click.option('--model', type=click.File('rb'), help='The trained model file')
@click.option('--train_loader', type=click.File('rb'), help='The training data loader file')
@click.option('--cost', type=click.File('rb'), help='The cost function file')
@click.option('--optimizer', type=click.File('rb'), help='The optimizer file')
@click.option('--epoch', type=int, default=10, help='The number of iterations over the entire training data')
def cli(model, train_loader, cost, optimizer, epoch):
    """Train a machine learning model."""
    # Load the model, train_loader, cost function, and optimizer from the files
    model = pickle.load(model)
    train_loader = pickle.load(train_loader)
    cost = pickle.load(cost)
    optimizer = pickle.load(optimizer)
    # Call the train function
    train(model, train_loader, cost, optimizer, epoch) 

In [4]:
def test(model, test_loader):
    """
    This function tests the accuracy of a given neural network model on a test dataset.

    Arguments:
        model (nn.Module): The model to be tested.
        test_loader (DataLoader): The test data in the form of PyTorch DataLoader.

    Returns:
        None
    """
    # Set the model to evaluation mode
    model.eval()
    # Initialize the number of correct predictions to 0
    correct = 0
    # Turn off gradient computation to speed up evaluation
    with torch.no_grad():
        # Loop through each batch in the test_loader
        for data, target in test_loader:
            # Reshape the data tensor to have shape (batch_size, -1)
            data = data.view(data.shape[0], -1)
            # Get the predictions from the model
            output = model(data)
            # Find the index of the maximum prediction for each sample in the batch
            pred = output.argmax(dim=1, keepdim=True)
            # Increase the number of correct predictions
            correct += pred.eq(target.view_as(pred)).sum().item()
    # Print the test accuracy
    print(f'Test set: Accuracy: {correct}/{len(test_loader.dataset)} = {100*(correct/len(test_loader.dataset))}%)')



In [ ]:
import pickle
import click

@click.command()
@click.option('--model', type=click.File('rb'), help='The trained model file')
@click.option('--train_loader', type=click.File('rb'), help='The training data loader file')
def cli(model, train_loader):
    """Train a machine learning model."""
    # Load the model, train_loader from the files
    model = pickle.load(model)
    train_loader = pickle.load(train_loader)
    # Call the test function
    test(model, train_loader) 

In [10]:
import click
import torchvision.transforms as transforms

# @click.command()
def training_transform():
    """
    This function returns a compose of data transforms for the training data.

    Returns:
        transforms.Compose: A compose of data transforms to be applied to the training data.
    """
    return transforms.Compose([
        # Randomly flip the image horizontally with a probability of 0.5 (Data Augumentation)
        transforms.RandomHorizontalFlip(p=0.5),
        # Convert the image to a PyTorch tensor
        transforms.ToTensor(),
        # Normalize the image tensor with mean (0.1307,) and standard deviation (0.3081,)
        transforms.Normalize((0.1307,), (0.3081,))
    ])

In [11]:
import click
import torchvision.transforms as transforms

# @click.command()
def testing_transform():
  """
    Define a transformation for the test data.
    
    Returns:
    A PyTorch `transforms.Compose` object.
  """
  return transforms.Compose([
    # Convert the image to a PyTorch tensor
    transforms.ToTensor(),
    # Normalize the image tensor with mean (0.1307,) and standard deviation (0.3081,)
    transforms.Normalize((0.1307,), (0.3081,))
  ])


In [13]:
# Set Hyperparameters
batch_size = 64
epoch = 10

# Download and load the training data
trainset = datasets.MNIST('data/', download=True, train=True, transform=training_transform())
testset = datasets.MNIST('data/', download=True, train=False, transform=testing_transform())
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


# Size of the input 
input_size = 784

# Number of classes
output_size = 10

# Create model
model = create_model(input_size, output_size)

# Cost function using Negative Log Likelihood Loss
cost = nn.NLLLoss()

# Stochastic Gradient Descent optimizer with learning rate 0.001 and momentum 0.9
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model for the specified number of epochs
train(model, train_loader, cost, optimizer, epoch)

# Evaluate the model on the test data
test(model, test_loader)

Epoch 0: Loss 0.01602686196565628, Accuracy 70.30833333333332%
Epoch 1: Loss 0.007675544358789921, Accuracy 84.65166666666667%
Epoch 2: Loss 0.006209364160895348, Accuracy 87.88666666666667%
Epoch 3: Loss 0.005043025128543377, Accuracy 90.39500000000001%
Epoch 4: Loss 0.004245693329721689, Accuracy 92.15%
Epoch 5: Loss 0.0037167943082749844, Accuracy 93.08333333333333%
Epoch 6: Loss 0.0033402012195438147, Accuracy 93.68666666666667%
Epoch 7: Loss 0.003047898644581437, Accuracy 94.24%
Epoch 8: Loss 0.002804572694003582, Accuracy 94.68333333333334%
Epoch 9: Loss 0.00259191426448524, Accuracy 95.15333333333334%
Test set: Accuracy: 9488/10000 = 94.88%)
